In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re



In [2]:
df=pd.read_csv("../raw_data/train.csv").sample(1000)

/tmp/ipykernel_2870/1386480518.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("../raw_data/train.csv").sample(1000)


In [3]:
df.head(5)

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
13910,0x6780,CUS_0xa53,July,NaN,36,243-69-8365,Musician,107289.36000000002_,9031.780000,6,...,Standard,815.62,24.059178,23 Years and 3 Months,No,207.777729,775.0864123247052,Low_spent_Medium_value_payments,200.3138587805253,Standard
6267,0x3ab9,CUS_0x36f9,April,Lucy Hornbyr,46,992-38-5349,_______,82065.16,6771.763333,5,...,Good,106.83,28.173288,NaN,No,180.340607,270.3058977300451,Low_spent_Medium_value_payments,506.52982894368745,Good
32226,0xd2d4,CUS_0x6b9d,March,Victoriat,45,390-39-5965,Doctor,9697.29,861.107500,4,...,Standard,530.96,28.423791,8 Years and 3 Months,Yes,28.774536,14.616968155217538,High_spent_Medium_value_payments,292.7192457831213,Standard
31476,0xce6e,CUS_0x2855,May,Berniez,54_,406-13-8367,Musician,34185.14,2659.761667,3,...,_,1461.18,38.591521,28 Years and 5 Months,No,103.401570,82.76519747897143,High_spent_Small_value_payments,339.8093994019825,Poor
69097,0x1aadf,CUS_0x98f1,February,Paritosh Bansaly,39,508-24-8627,Doctor,15301.71,NaN,5,...,Standard,1963.92,39.221072,17 Years and 8 Months,Yes,47.544028,29.924341164984888,Low_spent_Medium_value_payments,311.2458808057554,Poor


In [4]:
df.isnull().sum()

ID                            0
Customer_ID                   0
Month                         0
Name                        115
Age                           0
SSN                           0
Occupation                    0
Annual_Income                 0
Monthly_Inhand_Salary       152
Num_Bank_Accounts             0
Num_Credit_Card               0
Interest_Rate                 0
Num_of_Loan                   0
Type_of_Loan                105
Delay_from_due_date           0
Num_of_Delayed_Payment       59
Changed_Credit_Limit          0
Num_Credit_Inquiries         19
Credit_Mix                    0
Outstanding_Debt              0
Credit_Utilization_Ratio      0
Credit_History_Age           88
Payment_of_Min_Amount         0
Total_EMI_per_month           0
Amount_invested_monthly      56
Payment_Behaviour             0
Monthly_Balance               7
Credit_Score                  0
dtype: int64

In [5]:
#removing any negative or positive 4digit number replacing it to Nan and removing underscore at the end and replacing it with it initial value
def process_age(df, col):
    
    df[col] = df[col].str.replace('_', '')
    df[col] = df[col].str.replace('-', '')
    df[col] = df[col].str.replace(r'\b-\d{3}\b', '',regex=True)
    df[col] = df[col].str.replace(r'\b\d{3}\b', '',regex=True)
    df[col] = df[col].str.replace(r'\b-\d{4}\b', '',regex=True)
    df[col] = df[col].str.replace(r'\b\d{4}\b', '',regex=True)
    df[col] = df[col].replace('', np.nan)
    df[col] = df[col].astype(float)
    
    grouped_df = df.groupby('Customer_ID')
    mode_per_group = grouped_df[col].transform(lambda x: x.mode().iat[0] if len(x.mode()) > 0 else np.nan)
    df[col].fillna(mode_per_group, inplace=True)
    
    return df

In [6]:
clean_age_df = process_age(df,'Age')


In [7]:
clean_age_df['Age'].unique()

array([36., 46., 45., 54., 39., 19., 38., 23., 51., 50., 52., 34., 32.,
       nan, 42., 41., 33., 22., 18., 24., 20., 14., 55., 30., 43., 17.,
       21., 31., 48., 47., 15., 35., 28., 37., 29., 16., 27., 44., 25.,
       53., 49., 40., 26., 56.])

In [8]:
def clean_occ(df, col, pattern):
    df[col] = df[col].str.replace(pattern, '')
    df[col] = df[col].replace('', np.nan)
    
    grouped_df = df.groupby('Customer_ID')
    mode_per_group = grouped_df[col].transform(lambda x: x.mode().iat[0] if len(x.mode()) > 0 else np.nan)
    df[col].fillna(mode_per_group, inplace=True)
    
    return df

In [9]:
cleaned_occ_df=clean_occ(clean_age_df,"Occupation","_______")

In [10]:
cleaned_occ_df["Occupation"].unique()

array(['Musician', nan, 'Doctor', 'Developer', 'Engineer',
       'Media_Manager', 'Mechanic', 'Teacher', 'Manager', 'Scientist',
       'Writer', 'Entrepreneur', 'Architect', 'Lawyer', 'Accountant',
       'Journalist'], dtype=object)

In [11]:
def clean_ai(df, col):
    
    df[col] = df[col].str.replace('_', '')
    df[col] = df[col].replace('', np.nan)    
    grouped_df = df.groupby('Customer_ID')
    mode_per_group = grouped_df[col].transform(lambda x: x.mode().iat[0])
    df[col].fillna(mode_per_group, inplace=True)
    
    return df

In [12]:
clean_ai_df=clean_ai(cleaned_occ_df,"Annual_Income")

In [13]:
clean_ai_df["Annual_Income"].unique()

array(['107289.36000000002', '82065.16', '9697.29', '34185.14',
       '15301.71', '66567.32', '35586.4', '33683.95', '29132.58',
       '18666.83', '42153.13', '18128.1', '43249.78', '76249.74',
       '9607.53', '50733.18000000001', '53467.47', '19013.285',
       '36926.92', '138395.0', '36154.22', '74780.48', '14264.46',
       '50552.34', '78006.32', '15131.6', '33189.63', '90592.02',
       '58599.48', '19403.405', '69476.14', '26939.95', '19858.97',
       '36104.82', '9345.955', '63658.26', '94529.82', '47275.96',
       '35064.5', '71267.18', '10976886.0', '31563.9', '15723.705',
       '16916.38', '86584.74', '9955.255', '175572.44', '15051.065',
       '7890.515', '88165.12', '10737.365', '16718.08', '66978.24',
       '31183.16', '40023.81', '52882.77', '56089.95', '128066.44',
       '100956.21', '10216.36', '23957.75', '108472.35', '22418.63',
       '39706.44', '31429.85', '16734.13', '14180.48', '29377.62',
       '81769.35', '138154.96', '9379.93', '27268.7', '9737.955

In [14]:
clean_ai_df.columns

Index(['ID', 'Customer_ID', 'Month', 'Name', 'Age', 'SSN', 'Occupation',
       'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts',
       'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Type_of_Loan',
       'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit',
       'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt',
       'Credit_Utilization_Ratio', 'Credit_History_Age',
       'Payment_of_Min_Amount', 'Total_EMI_per_month',
       'Amount_invested_monthly', 'Payment_Behaviour', 'Monthly_Balance',
       'Credit_Score'],
      dtype='object')

In [15]:
clean_ai_df["Monthly_Inhand_Salary"].unique()

array([ 9031.78      ,  6771.76333333,   861.1075    ,  2659.76166667,
                  nan,  5800.27666667,  3076.99583333,  2559.715     ,
        1628.56916667,  3377.76083333,  1729.675     ,  3538.14833333,
        6299.145     ,   982.6275    ,  4402.765     ,  4461.6225    ,
        1358.08483948,  3081.24333333, 11619.91666667,  3043.85166667,
        6340.70666667,  1091.705     ,  4197.695     ,  1505.96666667,
        7433.335     ,  1904.95041667,  6016.67833333,  2466.99583333,
        1663.91416667,  3294.735     ,   558.82958333,  5281.855     ,
        8153.485     ,  4238.66333333,  2949.04166667,  6167.93166667,
        1513.37666667,  1406.69833333,  7228.395     ,   834.60458333,
       14422.03666667,  1156.25541667,   932.54291667,  1617.17333333,
        5473.52      ,  2760.59666667,  3214.3175    ,  4455.8975    ,
        4875.1625    , 10394.20333333,   679.36333333,  9233.3625    ,
        2136.21916667,  3387.87      ,  2879.15416667,  1429.51083333,
      

In [16]:
def clean_mis(df, col):
    grouped_df = df.groupby('Customer_ID')
    mode_per_group = grouped_df[col].transform(lambda x: x.mode().iat[0]if len(x.mode()) > 0 else np.nan)
    df[col].fillna(mode_per_group, inplace=True)
    
    return df

In [17]:
clean_mis_df=clean_mis(clean_ai_df,"Monthly_Inhand_Salary")

In [18]:
clean_mis_df.isnull().sum()

ID                            0
Customer_ID                   0
Month                         0
Name                        115
Age                          32
SSN                           0
Occupation                   62
Annual_Income                 0
Monthly_Inhand_Salary       146
Num_Bank_Accounts             0
Num_Credit_Card               0
Interest_Rate                 0
Num_of_Loan                   0
Type_of_Loan                105
Delay_from_due_date           0
Num_of_Delayed_Payment       59
Changed_Credit_Limit          0
Num_Credit_Inquiries         19
Credit_Mix                    0
Outstanding_Debt              0
Credit_Utilization_Ratio      0
Credit_History_Age           88
Payment_of_Min_Amount         0
Total_EMI_per_month           0
Amount_invested_monthly      56
Payment_Behaviour             0
Monthly_Balance               7
Credit_Score                  0
dtype: int64

In [19]:
clean_mis_df["Num_Bank_Accounts"].unique()

array([   6,    5,    4,    3,    7,    2,    9,    1,    8,   10,    0,
        871, 1018,  794,  771,  610, 1430,  870,  521,   59, 1516,  826,
       1656, 1641])

In [20]:
def clean_nba(df, col):
    df[col] = df[col].apply(lambda x: re.sub(r'\b\d{2}\b', '', str(x)))
    df[col] = df[col].apply(lambda x: re.sub(r'\b\d{3}\b', '', str(x)))
    df[col] = df[col].apply(lambda x: re.sub(r'\b\d{4}\b', '', str(x)))
    df[col] = df[col].replace('', np.nan)
    # df[col] = df[col].astype(float)
    grouped_df = df.groupby('Customer_ID')
    mode_per_group = grouped_df[col].transform(lambda x: x.mode().iat[0]if len(x.mode()) > 0 else np.nan)
    df[col].fillna(mode_per_group, inplace=True)

    return df

In [21]:
clean_nba_df=clean_nba(clean_mis_df,"Num_Bank_Accounts")

In [22]:
clean_nba_df["Num_Bank_Accounts"].unique()

array(['6', '5', '4', '3', '7', '2', '9', '1', '8', nan, '0'],
      dtype=object)

In [23]:
clean_nba_df["Num_Credit_Card"].unique()

array([   4,    5,    3,    7,   10,    2,    8,    9,    6,    1,   70,
       1452,   86,  367,   48,  629,  106,  575,  222, 1025, 1194, 1165,
        875,  294,  930,  277,  659,  380,  308,  822,  768, 1131])

In [24]:
def clean_nbc(df, col):
    df[col] = df[col].apply(lambda x: re.sub(r'\b\d{2}\b', '', str(x)))
    df[col] = df[col].apply(lambda x: re.sub(r'\b\d{3}\b', '', str(x)))
    df[col] = df[col].apply(lambda x: re.sub(r'\b\d{4}\b', '', str(x)))
    df[col] = df[col].replace('', np.nan)
    # df[col] = df[col].astype(float)
    grouped_df = df.groupby('Customer_ID')
    mode_per_group = grouped_df[col].transform(lambda x: x.mode().iat[0]if len(x.mode()) > 0 else np.nan)
    df[col].fillna(mode_per_group, inplace=True)

    return df

In [25]:
clean_nbc_df=clean_nbc(clean_nba_df,'Num_Credit_Card')

In [26]:
clean_nbc_df['Num_Credit_Card'].unique()

array(['4', '5', '3', '7', nan, '2', '8', '9', '6', '1'], dtype=object)

In [27]:
clean_nbc_df['Interest_Rate'].unique()

array([   7,    3,    9,    5,   20,    6,   19,   24,   27,   10,   12,
         22,   34,   17,   18,   15,   32,   14,   23,    8,   30,    1,
         16,    4,   33,   28,   25,    2,   21,   13,   29, 2120,   26,
       1680,   11,  696,   31, 3901, 3831,   76,  454, 5446, 5312, 2822,
       4033, 2098, 2410, 4146, 1245, 2302, 4219, 2610])

In [28]:
def clean_ir(df, col):
    df[col] = df[col].apply(lambda x: re.sub(r'\b\d{3}\b', '', str(x)))
    df[col] = df[col].apply(lambda x: re.sub(r'\b\d{4}\b', '', str(x)))
    df[col] = df[col].replace('', np.nan)
    # df[col] = df[col].astype(float)
    grouped_df = df.groupby('Customer_ID')
    mode_per_group = grouped_df[col].transform(lambda x: x.mode().iat[0]if len(x.mode()) > 0 else np.nan)
    df[col].fillna(mode_per_group, inplace=True)

    return df

In [29]:
clean_ir_df=clean_ir(clean_nbc_df,'Interest_Rate')

In [30]:
clean_ir_df['Interest_Rate'].unique()

array(['7', '3', '9', '5', '20', '6', '19', '24', '27', '10', '12', '22',
       '34', '17', '18', '15', '32', '14', '23', '8', '30', '1', '16',
       '4', '33', '28', '25', '2', '21', '13', '29', nan, '26', '11',
       '31', '76'], dtype=object)

In [31]:
clean_ir_df['Num_of_Loan'].unique()

array(['3', '4', '2', '6', '9', '5', '7', '1', '2_', '-100', '0', '1_',
       '8', '3_', '4_', '5_', '7_', '0_', '6_', '848', '31', '995', '8_',
       '9_', '1015'], dtype=object)

In [32]:
def clean_nol(df, col):
    
    df[col] = df[col].str.replace('_', '')
    df[col] = df[col].str.replace('-', '')
    df[col] = df[col].str.replace(r'\b-\d{3}\b', '',regex=True)
    df[col] = df[col].str.replace(r'\b\d{3}\b', '',regex=True)
    df[col] = df[col].str.replace(r'\b-\d{4}\b', '',regex=True)
    df[col] = df[col].str.replace(r'\b\d{4}\b', '',regex=True)
    df[col] = df[col].replace('', np.nan)
    df[col] = df[col].astype(float)
    
    grouped_df = df.groupby('Customer_ID')
    mode_per_group = grouped_df[col].transform(lambda x: x.mode().iat[0] if len(x.mode()) > 0 else np.nan)
    df[col].fillna(mode_per_group, inplace=True)
    
    return df

In [33]:
clean_nol_df=clean_nol(clean_ir_df,"Num_of_Loan")

In [34]:
clean_nol_df['Num_of_Loan'].unique()

array([ 3.,  4.,  2.,  6.,  9.,  5.,  7.,  1., nan,  0.,  8., 31.])

In [35]:
clean_nol_df['Type_of_Loan'].unique()



array(['Mortgage Loan, Personal Loan, and Mortgage Loan',
       'Not Specified, Home Equity Loan, and Auto Loan',
       'Not Specified, Credit-Builder Loan, Debt Consolidation Loan, and Personal Loan',
       'Credit-Builder Loan, Not Specified, Student Loan, and Debt Consolidation Loan',
       'Credit-Builder Loan, Student Loan, Student Loan, and Student Loan',
       'Mortgage Loan, and Not Specified',
       'Personal Loan, Auto Loan, Student Loan, Student Loan, Credit-Builder Loan, and Credit-Builder Loan',
       'Payday Loan, Not Specified, Personal Loan, and Student Loan',
       'Credit-Builder Loan, Auto Loan, Not Specified, Personal Loan, Debt Consolidation Loan, and Personal Loan',
       'Debt Consolidation Loan, Credit-Builder Loan, Auto Loan, Credit-Builder Loan, Payday Loan, Credit-Builder Loan, Mortgage Loan, Debt Consolidation Loan, and Home Equity Loan',
       'Payday Loan, Not Specified, and Payday Loan',
       'Home Equity Loan, Home Equity Loan, Auto Loan, and

In [36]:
def clean_tpo(df, col):
    grouped_df = df.groupby('Customer_ID')
    mode_per_group = grouped_df[col].transform(lambda x: x.mode().iat[0] if len(x.mode()) > 0 else np.nan)
    df[col].fillna(mode_per_group, inplace=True)
    
    return df

In [37]:
clean_tpo_df=clean_tpo(clean_nol_df,'Type_of_Loan')

In [38]:
clean_tpo_df['Type_of_Loan'].unique()


array(['Mortgage Loan, Personal Loan, and Mortgage Loan',
       'Not Specified, Home Equity Loan, and Auto Loan',
       'Not Specified, Credit-Builder Loan, Debt Consolidation Loan, and Personal Loan',
       'Credit-Builder Loan, Not Specified, Student Loan, and Debt Consolidation Loan',
       'Credit-Builder Loan, Student Loan, Student Loan, and Student Loan',
       'Mortgage Loan, and Not Specified',
       'Personal Loan, Auto Loan, Student Loan, Student Loan, Credit-Builder Loan, and Credit-Builder Loan',
       'Payday Loan, Not Specified, Personal Loan, and Student Loan',
       'Credit-Builder Loan, Auto Loan, Not Specified, Personal Loan, Debt Consolidation Loan, and Personal Loan',
       'Debt Consolidation Loan, Credit-Builder Loan, Auto Loan, Credit-Builder Loan, Payday Loan, Credit-Builder Loan, Mortgage Loan, Debt Consolidation Loan, and Home Equity Loan',
       'Payday Loan, Not Specified, and Payday Loan',
       'Home Equity Loan, Home Equity Loan, Auto Loan, and

In [39]:
clean_tpo_df.columns


Index(['ID', 'Customer_ID', 'Month', 'Name', 'Age', 'SSN', 'Occupation',
       'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts',
       'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Type_of_Loan',
       'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit',
       'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt',
       'Credit_Utilization_Ratio', 'Credit_History_Age',
       'Payment_of_Min_Amount', 'Total_EMI_per_month',
       'Amount_invested_monthly', 'Payment_Behaviour', 'Monthly_Balance',
       'Credit_Score'],
      dtype='object')

In [40]:
clean_tpo_df['Num_of_Delayed_Payment'].unique()


array(['9', '3764', '13', '14', '17', '21', '16', '10', nan, '2', '11',
       '20', '7', '18', '15', '12', '19', '1', '22_', '16_', '22', '23',
       '14_', '3', '18_', '6', '8', '3522', '0', '25', '1636', '5', '24',
       '4', '23_', '6_', '121', '-1', '15_', '8_', '27', '7_', '26',
       '3855', '13_', '2297', '-3', '17_', '4_', '1180', '24_', '290',
       '848_', '21_', '210', '-2', '11_', '1820', '9_'], dtype=object)

In [41]:
def clean_ndp(df, col):
    
    df[col] = df[col].str.replace('_', '')
    df[col] = df[col].str.replace('-', '')
    df[col] = df[col].str.replace(r'\b-\d{3}\b', '',regex=True)
    df[col] = df[col].str.replace(r'\b\d{3}\b', '',regex=True)
    df[col] = df[col].str.replace(r'\b-\d{4}\b', '',regex=True)
    df[col] = df[col].str.replace(r'\b\d{4}\b', '',regex=True)
    df[col] = df[col].replace('', np.nan)
    df[col] = df[col].astype(float)
    
    grouped_df = df.groupby('Customer_ID')
    mode_per_group = grouped_df[col].transform(lambda x: x.mode().iat[0] if len(x.mode()) > 0 else np.nan)
    df[col].fillna(mode_per_group, inplace=True)
    
    return df

In [42]:
  clean_ndp_df=clean_ndp(clean_tpo_df,'Num_of_Delayed_Payment')

In [43]:
clean_ndp_df['Num_of_Delayed_Payment'].unique()


array([ 9., nan, 13., 14., 17., 21., 16., 10.,  2., 11., 20.,  7., 18.,
       15., 12., 19.,  1., 22., 23.,  3.,  6.,  8.,  0., 25.,  5., 24.,
        4., 27., 26.])

In [44]:
clean_ndp_df['Num_Credit_Inquiries'].unique()

array([5.000e+00, 1.000e+00, 8.000e+00, 2.000e+00, 1.000e+01, 7.000e+00,
       1.200e+01, 4.000e+00, 6.000e+00, 1.400e+01, 9.000e+00, 1.100e+01,
             nan, 3.000e+00, 0.000e+00, 1.600e+01, 7.100e+02, 1.300e+01,
       1.700e+01, 5.140e+02, 1.850e+02, 1.500e+01, 2.244e+03, 6.150e+02,
       1.099e+03, 3.040e+02, 5.300e+02, 1.914e+03, 5.890e+02, 1.675e+03,
       1.575e+03])

In [45]:
def clean_ncq(df, col):
    grouped_df = df.groupby('Customer_ID')
    mode_per_group = grouped_df[col].transform(lambda x: x.mode().iat[0]if len(x.mode()) > 0 else np.nan)
    df[col].fillna(mode_per_group, inplace=True)

    return df

In [46]:
clean_ncq_df=clean_ncq(clean_ndp_df,"Num_Credit_Inquiries")

In [47]:
clean_ncq_df['Credit_Mix'].unique()

array(['Standard', 'Good', '_', 'Bad'], dtype=object)

In [50]:
def clean_cm(df, col):
    
    df[col] = df[col].str.replace('_', '')
    df[col] = df[col].str.replace('-', '')
    df[col] = df[col].replace('', np.nan)    
    grouped_df = df.groupby('Customer_ID')
    mode_per_group = grouped_df[col].transform(lambda x: x.mode().iat[0] if len(x.mode()) > 0 else np.nan)
    df[col].fillna(mode_per_group, inplace=True)
    
    return df

In [51]:
clean_cm_df=clean_cm(clean_ncq_df,'Credit_Mix')

In [52]:
clean_cm_df['Credit_Mix'].unique()

array(['Standard', 'Good', nan, 'Bad'], dtype=object)

In [ ]:
clean_cm_df['Outstanding_Debt'].unique()




In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)


In [ ]:
median_impute_cols = ['monthly_inhand_salary', 'num_of_delayed_payment', 'num_credit_inquiries',
                      'amount_invested_monthly', 'monthly_balance']
most_frequent_impute_cols = ['credit_mix', 'type_of_loan']
one_hot_encode_cols = ['occupation', 'type_of_loan', 'payment_behaviour']
ordinal_encode_cols = ['payment_of_min_amount','credit_mix']
min_max_scale_cols = ['age', 'num_bank_accounts', 'num_credit_card', 'num_credit_inquiries',
                      'num_of_delayed_payment']
standard_scale_cols = ['credit_utilization_ratio', 'changed_credit_limit']
robust_scale_cols = ['annual_income', 'monthly_inhand_salary', 'interest_rate', 'num_of_loan',
                     'delay_from_due_date', 'outstanding_debt', 'total_emi_per_month',
                     'amount_invested_monthly', 'monthly_balance']

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler, StandardScaler, RobustScaler, LabelEncoder

column_transformations = [
    ('impute_median', SimpleImputer(strategy='median'), median_impute_cols),
    ('impute_most_frequent', SimpleImputer(strategy='most_frequent'), most_frequent_impute_cols),
    ('one_hot_encode', OneHotEncoder( sparse=False,handle_unknown='ignore'), one_hot_encode_cols),
    ('ordinal_encode', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), ordinal_encode_cols)
]

scaling_transformer = [('min_max_scale', MinMaxScaler(), min_max_scale_cols),
                       ('standard_scale', StandardScaler(), standard_scale_cols),
                       ('robust_scale', RobustScaler(), robust_scale_cols)]

preprocessor = ColumnTransformer(transformers=column_transformations +
                                               scaling_transformer)

pipeline = make_pipeline(preprocessor)

X_train_preprocessed = pipeline.fit_transform(X_train)
X_test_preprocessed = pipeline.transform(X_test)




In [ ]:
label_encoder = LabelEncoder()
y_train_transformed= label_encoder.fit_transform(y_train)
y_test_transformed=label_encoder.transform(y_test)
